In [1]:
from surprise import accuracy, Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [3]:
import pandas as pd
import pathlib
DATA_DIR = pathlib.Path().resolve().parent / "data"
print(DATA_DIR)
DATA_DIR.exists()

C:\Users\USER\Documents\recommender\src\data


True

In [4]:
dataset = DATA_DIR / 'ratings_small.csv'
dataset.exists()

True

In [5]:
df = pd.read_csv(dataset)
df['rating'].dropna(inplace=True)
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
df.rating.max(), df.rating.min()

(5.0, 0.5)

In [7]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [8]:
algo = SVD(verbose=True, n_epochs=20)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=4, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing

{'test_rmse': array([0.89652163, 0.89554446, 0.90548938, 0.89939129]),
 'test_mae': array([0.6896917 , 0.6896856 , 0.69781217, 0.69341721]),
 'fit_time': (2.174623966217041,
  2.059000015258789,
  2.385711431503296,
  2.1390225887298584),
 'test_time': (0.39490294456481934,
  0.4878859519958496,
  0.32563352584838867,
  0.4422783851623535)}

In [9]:
trainset = data.build_full_trainset()
algo.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [10]:
testset = trainset.build_testset()
predictions = algo.test(testset)
#RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)

RMSE: 0.6423


0.6423280163426627

In [19]:
sample_row = df.sample(n=1)
userId = sample_row['userId'].values[0]
movieId = sample_row['movieId'].values[0]
print(userId, movieId)

547 50


In [14]:
pred = algo.predict(uid=userId, iid=movieId)
pred.est

2.6986356559611226

In [20]:
import pickle


In [27]:
algo_data = {"model": algo}
with open('model.pkl', 'wb') as f:
    pickle.dump(algo_data, f)

In [28]:
model_algo= None
with open('model.pkl', 'rb') as f:
    model_data_loaded = pickle.load(f)
    model_algo = model_data_loaded.get('model')

In [29]:
model_algo.predict(uid=userId, iid=movieId).est


4.454125570544983

In [30]:
sample_rows = df.sample(n=10).to_dict('records')
for row in sample_rows:
    userId = row['userId']
    movieId = row['movieId']
    pred = model_algo.predict(uid=userId, iid=movieId).est
    print(userId, movieId, pred)
    

564 1956 4.658764861978388
547 52967 3.7370260858303577
481 4306 3.755414182494385
597 451 4.162339826252978
547 253 2.2771128980536965
306 480 3.6955108244330415
102 858 4.799418238248334
63 1961 3.802464599846777
22 4720 3.42410582043006
327 2688 3.748833076518154
